# NumPy: Работаем с численными данными эффективно

Во время работы с астрономическими данными очень часто нужно взаимодействовать с большими наборами чисел - список звездных величин, матрица значений пикселей в ПЗС-кадре, массивы результатов симуляций, и др.

Хотя встроенные в Python списки достаточно гибкие, они не очень эффективны для проделывания математических операций на больших объемах чисел. Тут как раз и приходит на помощь библиотека **NumPy** (произносится как *"Нам-Пай"*).

NumPy - фундаментальная библиотека Python, предоставляющая основные инструменты для работы с численными массивами. С помощью неё численные расчеты становятся быстрыми и легкими для написания, в сравнении со стандартными списками Python и работой в циклах. Огромное множество других научных библиотек Python, вроде Pandas, Matplotlib или Astropy, построены с Numpy в своей основе.

В этом ноутбуке рассмотрим ключевые понятия NumPy и продемонстрируем, как с помощью них обрабатывать астрономические данные более эффективно.

**Задачи:**

* Сравнить стандартные методы обработки массивов с методами NumPy.
*   Изучить ключевой объект NumPy: `ndarray`.
*   Создать массивы NumPy разными способами.
*   Посмотреть на свойства массивов NumPy (форма, размер, тип данных).
*   Выбрать и изменить часть массива с помощью индексации и срезов.
*   Произвести базовые математические операции над целыми массивами.
*   Использовать общепринятые функции NumPy для вычислений.
*   Рассмотреть примеры использования NumPy в астрономии.

**Ноутбук предполагает знания основного синтаксиса Python (переменные, типы данных, списки, циклы)**

**Замечание**
Запустите ячейку ниже по `SHIFT+ENTER` для установки нужных пакетов, если вы не уверены, что такие установлены. Если они уже есть, её можно пропустить.

In [ ]:
!pip install numpy pandas matplotlib

## Зачем Numpy?
Пусть у нас есть набор звездных величин `star_brightnesses`. Сгенерируем большие данные с помощью библиотеки random:

In [ ]:
from random import Random

r = Random("42") # задаем для генератора псевдослучайных чисел базовое значение (сид), чтобы результаты воспроизводились
print(r.uniform(5, 20)) # проверка: для сида 42 "случайное" число из диапазона от 5 до 20 равно 12.104891328789904

In [ ]:
n = 10 ** 7
star_brightnesses = [r.uniform(5, 20) for _ in range(n)] # создадим тысячу случайных значений звездных величин

Положим, мы хотим пересчитать звездные величины в поток в единицах потока звезды с $m=0$ по формуле Погсона:
$$\frac{F}{F_0} = 10^{-0.4 m}$$
Со списками это делается примерно так:

In [ ]:
%%timeit -n 1 -r 10  # магическая функция для подсчета времени выполнения
fluxes_list = []
for brightness in star_brightnesses:
    fluxes_list.append(10 ** (-0.4 * brightness ))

In [ ]:
print("Используем списки и цикл, первые 10 значений:", ' '.join(f"{i:.3e}" for i in fluxes_list[:10]))

В принципе сработало, но вот для больших объемов данных обработка циклом может быть медленной. Попробуйте заменить показатель степени в n на 5-6 и посмотреть на время выполнения.

Смотрим, как это делается в Numpy:

In [ ]:
import numpy as np # Стандартное сокращение для Numpy
# Превращаем список в массив Numpy
star_brightnesses_np = np.array(star_brightnesses)

In [ ]:
%%timeit -n 1 -r 5

# Используем математическую операцию на всем массиве
fluxes_np = 10 ** (-0.4 * star_brightnesses_np)

In [ ]:
print("Через NumPy:", ' '.join(f"{i:.3e}" for i in fluxes_np[:10])) # проверим, что посчиталось то же самое

Способность использовать целые массивы в качестве аргументов функций возможна благодаря `векторизации` всего массива, и это одна из причин, почему Numpy эффективен для таких вычислений.

## `ndarray`: ключевой тип данных в NumPy

`ndarray` ("N-dimensional array", N-мерный массив) это таблица значений одного типа данных. Можно представлять себе её в виде матрицы (2 измерения) или даже куба (три измерения) чисел.

Массив `ndarray` является:
*   **Однородным:** Все элементы массива должны быть одного типа данных (все целые, все типа числа с плавающей точкой). Это ключевое различие со списками обеспечивает массивам NumPy их эффективность.
*   **N-мерным:** Может иметь любую размерность (1D для списка значений, 2D для таблица, 3D для куба изображения в разных полосах, и т.д.)

### Пример 1: создание массивов

In [ ]:
# 1. Из списка или кортежа Python::
# 1-мерный массив
star_magnitudes = [2.5, 1.8, 0.1, -1.4]
magnitude_array = np.array(star_magnitudes)
print("1D массив из списка:", magnitude_array)

# 2-мерный массив (таблица)
# Представление [RA, Dec] для трех звезд
star_data = [
    [101.28, -16.71], # Сириус
    [37.95, 89.26],   # Полярная
    [219.90, -60.83] # Альфа Центавра
]
star_array_2d = np.array(star_data)
print("\n2D массив из списка списков:")
print(star_array_2d)

In [ ]:
# 2. Используя встроенные генераторы NumPy:
# Массив из нулей (полезно для инициализации массива определенного размера)
zeros_array = np.zeros(5) # массив из 5 нулей
print("Массив нулей:", zeros_array)

# Массив единиц размерами 2x3
ones_array = np.ones((2, 3)) # форма представлена кортежом (число строк, число столбцов)
print("\nМассив из единиц (2x3):")
print(ones_array)

# Массив из последовательности целых чисел от 0 до 10 не включительно (аналог range)
arange_array = np.arange(10)
print("\nМассив из np.arange(10):", arange_array)

# Массив из 5 расставленным равномерно значений между 0 и 1 (включительно)
linspace_array = np.linspace(0, 1, 5)
print("\nМассив из linspace(0, 1, 5):", linspace_array)

## Аттрибуты массива: что он знает про самого себя

Массивы NumPy имеют несколько полезных аттрибутов, по которым можно узнать структуру и содержание массива.

*   `.shape`: Кортеж из размеров каждого измерения (число строк, число столбцов, и т.д.)
*   `.size`: Полное число элементов в массиве.
*   `.ndim`: Размерность массива (то есть 1 для одномерного, 2 для двумерного и т.д.).
*   `.dtype`: Тип данных, записанных в массив (например, `int64`, `float64`, число обозначает количество бит памяти, выделенное под каждое значение).

### Пример 2: аттрибуты 
Используем массивы, похожие на использованные ранее:

In [ ]:
magnitude_array = np.array([3, 2, 0, -1])
star_array_2d = np.array([[101.28, -16.71], [37.95, 89.26], [219.90, -60.83]])

print("--- Аттрибуты 1-мерного массива ---")
print("Представление массива:", magnitude_array)
print("Форма:", magnitude_array.shape)
print("Мощность:", magnitude_array.size)
print("Размерность:", magnitude_array.ndim)
print("Тип данных:", magnitude_array.dtype)

print("\n--- Аттрибуты 2-мерного массива ---")
print("Представление массива:")
print(star_array_2d)
print("Форма:", star_array_2d.shape) # (3, 2) означает 3 строки, 2 столбца
print("Мощность:", star_array_2d.size)
print("Размерность:", star_array_2d.ndim)
print("Тип данных:", star_array_2d.dtype) # Обратите внимание, что тип данных назначается автоматически

## Индексация и срезы

Прямо как в списках, можно обращаться к отдельным элементам массива, используя индексацию и срезы с аналогичным синтаксисом.

*   **Индексация:** Доступ к одному элементу, используя его положение (индекс). Помните, что индексация в Python начинается с нуля!
*   **Срезы:** Доступ к диапазону значений.

Для 2- и более мерных массивов вместо многократного написания квадратных скобочек можно использовать кортеж из индексов или срезов: `array[индекс_или_срез_по_строкам, индекс_или_срез_по_столбцам]`.

### Пример 3

In [ ]:
# добавим к звездам их параллаксы в угловых секундах, чтобы сделать таблицу 3x3
star_data = np.array([
    [101.28, -16.71, 0.380], # Строка 0 (Сириус)
    [37.95, 89.26, 0.007],   # Строка 1 (Полярная)
    [219.90, -60.83, 0.769]  # Строка 2 (Альфа Центавра)
])
print("Исходный массив:")
print(star_data)

print("\n--- Индексация ---")
# Обратимся к значению в строке 1, столбце 1 (склонение Полярной)
polaris_dec = star_data[1, 1]
print("Склонение Полярной:", polaris_dec)

In [ ]:
# Обратимся к значению в строке 0, столбце 0 (ПВ Сириуса)
sirius_ra = star_data[0, 0]
print("Прямое восхождение Сириуса:", sirius_ra)

# Обратимся ко всей строке (Строка 2 - Данные Альфы Центавра)
proxima_row = star_data[2, :] # ':' в качестве индекса означает "всё по этой размерности"
print("\nДанные Альфы Центавра (строка 2):", proxima_row)

# Обратимся ко всему столбцу (Столбец 1 - склонения всех звезд)
declination_column = star_data[:, 1]
print("Склонения всех звезд (столбец 1):", declination_column)


print("\n--- Срезы ---")
# Вытащим первые две строчки
first_two_rows = star_data[0:2, :] # От строчки 0 до строчки 2 не включительно
print("Первые две строчки:")
print(first_two_rows)

In [ ]:
# Получим 1 и 2 колонки (склонение и параллаксы у всех звезд)
dec_mag_columns = star_data[:, 1:3]
print("Склонение и параллаксы:")
print(dec_mag_columns)

# Получим прямое восхождение и склонение только для двух звезд:
first_two_stars_ra_dec = star_data[0:2, 0:2]
print("α и δ для первых двух звезд:")
print(first_two_stars_ra_dec)

In [ ]:
# Видоизменение элементов тоже можно делать по индексу
star_data[1, 2] = 0.003 # Изменим параллакс Полярной
print("Массив после обновления параллакса Полярной:")
print(star_data)

## Базовые операции над массивами

Вы уже увидели, что NumPy может проводить математические операции над всем массивом целиком. Каждая операция применяется поэлементно, либо же используется преобразование одного типа к другому, как в случае с умножением на одно число.

### Пример 4: арифметика массивов

In [ ]:
magnitudes = np.array([2.5, 1.8, 0.1, -1.4])
offsets = np.array([0.1, 0.1, 0.1, 0.1]) # массивы одного размера

# Сложение
shifted_magnitudes = magnitudes + offsets
print("Смещенный массив:", shifted_magnitudes)

# Вычитание массивов
subtracted_values = magnitudes - offsets
print("Смещенный массив в другую сторону:", subtracted_values)

In [ ]:
# Умножение
doubled_magnitudes = magnitudes * 2
print("Величины * 2:", doubled_magnitudes)

# Деление
divided_magnitudes = magnitudes / 4.0
print("Величины / 4:", divided_magnitudes)

# Возведение в степень
relative_fluxes = 10**(-0.4 * magnitudes)
print("Относительные потоки:", relative_fluxes)

### Пример 5: сравнение массивов

In [ ]:
magnitudes = np.array([2.5, 1.8, 0.1, -1.4, 3.0, 0.5])

# Найдем звезды ярче 1.0 величины
is_brighter_than_1 = magnitudes < 1.0
print("Величина < 1.0?", is_brighter_than_1) # вернет массив-маску из булевых значений

In [ ]:
# Массив-маску можно использовать для того, чтобы выделить только необходимые значения
bright_stars = magnitudes[is_brighter_than_1]
print("Величины меньше первой:", bright_stars)

In [ ]:
# Найдем звезды с величинами больше 0 И меньше 2
# Для каждого сравнения не забываем про скобки
is_between_0_and_2 = (magnitudes > 0) & (magnitudes < 2) # Используем '&' для поэлементного and
print("\nКакие элементы имеют величину между 0 и 2?", is_between_0_and_2)
print("Величины между 0 и 2:", magnitudes[is_between_0_and_2])

### Пример 6: функции NumPy

Большая часть функций копирует таковые из стандартной библиотеки и модуля `math`, поэтому импортировать его не обязательно при использовании NumPy:

In [ ]:
stellar_masses = np.array([0.8, 1.2, 0.5, 2.0, 0.9]) # массы набора звезд в солнечных массах

# Sum: подсчитаем полную массу звезд в массиве
total_mass = np.sum(stellar_masses)
print(f"Полная масса: {total_mass} M☉")

# Mean: подсчитаем среднюю массу звезд
average_mass = np.mean(stellar_masses)
print(f"Средняя масса: {average_mass} M☉")

In [ ]:
# Max и min: найти наибольший и наименьший элемент
max_mass = np.max(stellar_masses)
min_mass = np.min(stellar_masses)
print(f"Все массы находятся в диапазоне [{min_mass}, {max_mass}] M☉")

In [ ]:
# Тригонометрия и преобразование углов:
orbital_phases_radians = np.array([0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi]) # углы в радианах
print("Орбитальные фазы (радианы):", orbital_phases_radians)
orbital_phases_degrees = np.degrees(orbital_phases_radians)
print("Орбитальные фазы (градусы):", orbital_phases_degrees)
print("Косинус фазы:", np.cos(orbital_phases_radians)) # Поэлементный косинус
print("Синус фазы:", np.sin(orbital_phases_radians))   # Поэлементный синус

## Сохранение и загрузка массивов

Периодически большие массивы данных требуется записать в файл. Можно поступить двумя способами:

1.  **Сохранить массив в бинарный формат NumPy (`.npy`):** эффективно, быстро, занимает мало места, сохраняет тип данных и форму массива, однако, простым блокнотом такой файл не прочитать.
2.  **Сохранить массив в виде текстового файла (`.txt`, `.csv`, и проч.):** единственный плюс - универсальность и возможность прочтения человеком как текстовую таблицу со значениями, разделенными пробелами или запятыми. Однако, специфическая информация Numpy (например, тип данных) может теряться.

Посмотрим на примеры использования `np.save()`, `np.load()`, `np.savetxt()`, и `np.loadtxt()`.

### Пример 7: сохранение и загрузка массивов в формате `.npy`
Пусть имеется массив расстояний до звезд в световых годах:

In [ ]:
star_distances_ly = np.array([4.37, 8.60, 10.0, 14.5, 36.0])

### Сохранение в `.npy`
`np.save()` принимает в качестве аргументов имя файла (без `.npy`, оно добавляется автоматически) и название переменной-массива для сохранения:

In [ ]:
np.save("my_star_distances", star_distances_ly) # эта строчка создаст файл "my_star_distances.npy"
print("Массив сохранен в 'my_star_distances.npy'")

### Загрузка из `.npy`
`np.load()` принимает в качестве аргумента название файла (вместе с `.npy`):

In [ ]:
loaded_distances = np.load("my_star_distances.npy")
print("Массив загружен из 'my_star_distances.npy':")
print(loaded_distances)

In [ ]:
# Проверим, что массивы действительно одинаковые:
print("Тип загруженных данных:", type(loaded_distances))
print("Равны ли оригинал и загруженный из файла массивы?", np.array_equal(star_distances_ly, loaded_distances))

### Пример 8: сохранение и загрузка в текстовый файл (.txt или .csv)

Пусть у нас есть данные нескольких планет: `[Масса (в земных массах), Радиус (в земных радиусах), Орбитальный период (в днях)]`

In [ ]:
exoplanet_table = np.array([
    [1.0, 1.0, 365.25],   # Земля
    [0.11, 0.53, 687.0],  # Марс
    [317.8, 11.2, 4331.0] # Юпитер
])
print("Данные для сохранения:")
print(exoplanet_table)

### Сохранение в текстовый файл
`np.savetxt()` используется для текстового формата.

В качестве аргументов указывается имя файла, название массива, и опциональные аргументы (например, 'delimiter' задает символ-разделитель):

In [ ]:
np.savetxt("exoplanet_data.csv", exoplanet_table, delimiter=",") # в качестве разделителя используем запятую
print("Массив сохранен в 'exoplanet_data.csv'")

### Загрузка из текстового файла
`np.loadtxt()` - функция загрузки из текстового файла.

В качестве аргумента - имя файла, и опциональные аргументы, типа 'delimiter':

In [ ]:
loaded_exoplanet_data = np.loadtxt("exoplanet_data.csv", delimiter=",")
print("Загружено из 'exoplanet_data.csv':")
print(loaded_exoplanet_data)

# Проверим, что загрузилось то же самое:
print("Тип загруженных данных:", type(loaded_exoplanet_data))
print("Равны ли оригинал и загруженный из файла массивы?", np.array_equal(exoplanet_table, loaded_exoplanet_data))

# Можно открыть 'exoplanet_data.csv' в любом текстовом редакторе или даже отсюда, из Jupyter, чтобы убедиться в его читабельности

### Пример 9: Загрузка текстового файла с заголовком/комментариями

Часто, в текстовом файле помимо данных есть заголовок или комментарии с пояснениями. Можно воспользоваться функцией `np.genfromtxt()` и её параметрами, чтобы их пропустить.

Пусть у нас есть файл 'star_observations.txt':

```code
# Observation Data - 2023-10-27
# Star Name, Flux, Error
  Alpha, 123.45, 1.2
  Beta, 56.78, 0.8
  Gamma, 234.56, 2.1
```


In [ ]:
# Создадим файл с таким наполнением:
file_content = """# Observation Data - 2023-10-27
# Star Name, Flux, Error
Alpha, 123.45, 1.2
Beta, 56.78, 0.8
Gamma, 234.56, 2.1
"""

In [ ]:
with open("star_observations.txt", "w") as f:
    f.write(file_content)
print("Создан файл 'star_observations.txt'")

In [ ]:
# --- Загружаем данные, пропуская заголовок---
# используем skip_header
loaded_obs_data = np.genfromtxt(
    "star_observations.txt", 
    delimiter=",", 
    skip_header=2,       # same effect as skiprows
    dtype=None # чтобы считались и строки в том числе
)
print("\nЗагружены данные 'star_observations.txt' (первые 2 строки пропущены):")
print(loaded_obs_data)

# Замечание: loadtxt лучше использовать для чисто численных массивов. Для массивов с текстом лучше воспользоваться
# pandas read_csv (который использует внутри себя тот же NumPy)

**Дополнительные ссылки**

Документация Numpy: <https://numpy.org/doc/stable/user/index.html>

Краткое введение в Numpy: <https://numpy.org/doc/stable/user/absolute_beginners.html>

Яндекс-учебник: <https://education.yandex.ru/handbook/python/article/moduli-math-i-numpy>

## Упражнения

1. Создайте одномерный массив NumPy `planet_radii` с примерными радиусами планет земной группы в земных радиусах: `[0.38, 0.95, 1.0, 0.53]`.
2. Выведите `.shape`, `.size`, и `.ndim` для `planet_radii`.
3. Создайте новый массив `planet_diameters` умножением `planet_radii` на 2. Выведите массив `planet_diameters`.
4. Используя `planet_radii` создайте массив-маску из булевых значений, где `True` будет на тех индексах, где значение радиуса меньше 0.6 от земного. Используйте эту маску для вывода этих значений.
5. Выведите среднее значение радиуса из `planet_radii` встроенной в NumPy функцией.
6. Создайте двумерный массив NumPy `star_distances` размерами (50, 2). Заполните с помощью `np.random.uniform(0, 100, 50)` случайными числами, равномерно распределенными от 0 до 100, первую строку в этом массиве - пусть это будут расстояния до звезд в парсеках. Заполните вторую строку расстояниями до этих звезд в световых годах.
7. Из `star_distances` выведите 10 центральных значений расстояний в световых годах с помощью срезов.
8. Выведите наибольшее расстояние в парсеках из `star_distances`.
9. Сохраните `star_distance` в бинарном формате в файл `star_distances.npy`, и в текстовом с разделителем-запятой в файл `star_distance.csv`.
10. Загрузите содержимое файлов `star_distances.npy` и `star_distance.csv` в разные переменные-массивы. Найдите разницу между этими массивами и убедитесь, что она равна нулю.